In [17]:
import pandas as pd
import numpy as np
import csv
import ast

def process_csv(file_path):
    product_titles = []
    gcms_analysis = []
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            product_titles.append(row['Product Title'])
            gcms_analysis.append(ast.literal_eval(row['GCMS analysis']))  # Convert string to dictionary
            
    return product_titles, gcms_analysis
    
def process_gcms(data_dict, product_name):
    dfs = {}
    for key, value in data_dict.items():
        compound_dict = {}
        max_len = max(len(val[1]) for val in value)
        for compound, values in value:
            values.extend([np.nan] * (max_len - len(values)))  # Extend shorter lists with NaNs
            compound_dict[compound] = values
        compound_dict['Product Name'] = [product_name] * max_len
        df = pd.DataFrame(compound_dict)
        
        # Drop the 'Product Name' column before calculating the mean
        df_without_product = df.drop(columns=['Product Name'])
        
        # Calculate the mean for each compound
        mean_values = df_without_product.mean(axis=0)
        
        # Replace the rows with the mean values
        processed_df = pd.DataFrame(mean_values).T
        processed_df['Product Name'] = [product_name]
        
        dfs[key] = processed_df
    
    return dfs

def combine_dataframes(dfs1, dfs2):
    # Get the union of keys from dfs1 and dfs2
    all_keys = set(dfs1.keys()).union(dfs2.keys())

    # Combine the DataFrames
    combined_dfs = {}
    for key in all_keys:
        # Concatenate the DataFrames if both keys exist, otherwise choose the existing one and replace NaNs with 0
        if key in dfs1 and key in dfs2:
            combined_df = pd.concat([dfs1[key], dfs2[key]], ignore_index=True)
            combined_df.fillna(0, inplace=True)  # Replace NaNs with 0
            combined_dfs[key] = combined_df
        elif key in dfs1:
            combined_dfs[key] = dfs1[key].fillna(0)  # Replace NaNs with 0
        else:
            combined_dfs[key] = dfs2[key].fillna(0)  # Replace NaNs with 0
            
    return combined_dfs
    
# Define the data
file_path = 'aromatics_data.csv'
product_title, gcms_analysis = process_csv(file_path)

main_dfs = {}
temp_dfs = {}

main_dfs = process_gcms(gcms_analysis[0],product_title[0])

for i in range(len(product_title)-1):
    temp_dfs = process_gcms(gcms_analysis[i+1],product_title[i+1])
    main_dfs = combine_dataframes(main_dfs, temp_dfs)
    
for key, df in main_dfs.items():
    file_name = f"{key}.csv"
    df.to_csv(file_name, index=False)
    print(f"DataFrame '{key}' saved to {file_name}")


    

Combined DataFrame for KETONE:
     3-octanone  camphor  cryptone  carvone  \
0        1.2525   0.2550    0.1025   0.0375   
1        0.0000   0.0000    0.0000   0.0475   
2        0.0000   0.0000    0.0000   0.0075   
3        0.0000   0.0000    0.0000   0.0250   
4        0.0000   0.0000    0.0000   0.0000   
..          ...      ...       ...      ...   
133      0.0000   2.6675    0.0000   0.0000   
134      0.0000   0.3075    0.0000   0.0050   
135      0.2450   0.0000    0.0000   0.0000   
136      0.0000   9.7000    0.0000   0.0000   
137      0.0000   0.0000    0.0000   0.0000   

                           Product Name  thuja-2,4(10)-diene  verbenone  \
0                          Lavender Oil                0.000     0.0000   
1                      Frankincense Oil                0.425     0.2375   
2                        Peppermint Oil                0.000     0.0000   
3                      Orange Sweet Oil                0.000     0.0000   
4                            